### Create Encoder

* Basically, here is about what data information we want to save from the train dataset and will be applied to the new dataset. For example, encoder for the text and categorical variables or mean value for missing value imputation on numerical variable or the mean and std for the scaling. 
* This part should be able to connect seamlessly to the part about preparing new data for model to predict.

In [1]:
suppressMessages(library(dplyr))

# params
method = "mean"

In [2]:
# file and directory info
project_dir = '/Users/chou/Desktop/mlflow_dvc_cookiecutter/DSProjectTemplate'
interim_folder = "/data/interim/"
processed_folder = "/data/processed/"

# input
train = arrow::read_parquet(paste0(project_dir, interim_folder, "train.parquet"))
num_variables = train %>% select(-quality) %>% names(.)

# functions
Encoder_numeric = function(train, num_variables, method){
    train_num = train %>% select(!!!num_variables)
    
    summary_mean = train_num %>% summarise_at(num_variables, mean, na.rm = TRUE)
    summary_std = train_num %>% summarise_at(num_variables, sd, na.rm = TRUE)

    if (method == "mean") {
        summary_num = summary_mean
    }
    if (method == "median") {
         summary_num = train_num %>% summarise_at(num_variables, median, na.rm = TRUE)
    }
    
    summary_num_list = list()
    for (i in 1:ncol(summary_num)){
        col_name = names(summary_num)[i]
        summary_num_list[[i]] = data.frame(mean = as.numeric(summary_mean[,col_name]), 
                                           std = as.numeric(summary_std[,col_name]), 
                                           impute = as.numeric(summary_num[,col_name]))
        
    }
    names(summary_num_list) = names(summary_num)
    
    return(summary_num_list)
}

# output and save
encoder_num = Encoder_numeric(train, num_variables, method)
saveRDS(encoder_num, paste0(project_dir, processed_folder, "encoder_num.rds"))

In [3]:
encoder_num

mean,std,impute
6.844609,0.8374403,6.844609
mean,std,impute
0.2788963,0.1014861,0.2788963
mean,std,impute
0.333913,0.121522,0.333913
mean,std,impute
6.315365,5.045253,6.315365
mean,std,impute
0.04569485,0.02199551,0.04569485
mean,std,impute
